# Collect and convert results to Latex tables

In [5]:
import glob
import numpy as np

In [2]:
path = 'mobilenet_v3-cifar10.txt'.lower()
num_exams = "Number of examples in computing weights heuristically: ".lower()
error_msg = "A error occured".lower()
not_found = "not found from".lower()

model_version = "Model version:".lower()

def empty_res_obj():
  return {
    'subset_size': [],
  'h_train_loss': [],
  'h_train_acc': [],
  'h_test_loss': [],
  'h_test_acc': [],
  'gh_train_loss': [],
  'gh_train_acc': [],
  'gh_test_loss': [],
  'gh_test_acc': [],
  'g_train_loss': [],
  'g_train_acc': [],
  'g_test_loss': [],
  'g_test_acc': []}

def extract_values(line: str):
  arrs = line.split(' ')
  # [:-1] dropping ','
  return float(arrs[2][:-1]), float(arrs[4])

def collect_results(path):
  all_res = {}
  with open(path, 'r') as f:
    for line in f:
      line = line.lower()
      if model_version not in line: continue
      
      # get model version
      m_v = line[line.rfind(' ')+1:-1]
      res = empty_res_obj()
      all_res[m_v] = res
      for size in [64, 128, 256, 512, 1024, 2048]:
        # skip empty line
        f.readline()
        # get the subset size
        line = f.readline().lower()
        set_size = int(line[line.rfind(' ')+1:])
        res['subset_size'].append(set_size)
        # skips: shape
        f.readline()
        # heuristic info
        f.readline()

        line = f.readline().lower()
        # check is whether error or info
        if error_msg in line: continue
        line = f.readline().lower()
        if not_found in line: continue

        # get results: heuristics on train
        loss, acc = extract_values(line)
        res['h_train_acc'].append(acc)
        res['h_train_loss'].append(loss)

        # get results: heuristics on test
        loss, acc = extract_values(f.readline())
        res['h_test_acc'].append(acc)
        res['h_test_loss'].append(loss)

        # skip the next info line:
        # Training model on all heurstic weights with epochs 5
        f.readline()
        # get results: train on the heuristic weights
        line = f.readline()
        loss, acc = extract_values(line)
        res['gh_train_acc'].append(acc)
        res['gh_train_loss'].append(loss)

        # get results: train on the heuristic weights
        loss, acc = extract_values(f.readline())
        res['gh_test_acc'].append(acc)
        res['gh_test_loss'].append(loss)

        # skip the next info line:
        # Training model on all initial weights with epochs 5
        f.readline()
        # get results: train on the initial weights
        loss, acc = extract_values(f.readline())
        res['g_train_acc'].append(acc)
        res['g_train_loss'].append(loss)

        # get results: train on the initial weights
        loss, acc = extract_values(f.readline())
        res['g_test_acc'].append(acc)
        res['g_test_loss'].append(loss)
  
  return all_res

In [16]:
ds_name = 'mini_speech_commands'
all_files = glob.glob(f'*{ds_name}.txt')

for file in all_files:
  path = f'./{file}'
  all_res = collect_results(path)
  m_name = file.replace(f'{ds_name}.txt', '')[:-1]
  m_name = m_name.replace('_', '')
  for m_v, res in all_res.items():
    m_v = m_v.replace('classification', 'cl.')\
      .replace('feature-vector', 'fe.')\
      .replace('-fe', '-fe.')\
      .replace('..', '.')
    print(f'{m_name}({m_v})', end='')
    for tr_acc, te_acc in zip(res['h_train_acc'], res['h_test_acc']):
      print(f" & {tr_acc:.2f}/{te_acc:.2f}", end='')
    print(' \\\\')

inceptionv3(cl.) & 0.15/0.16 & 0.14/0.13 & 0.16/0.13 & 0.17/0.16 & 0.18/0.15 & 0.21/0.20 \\
inceptionv3(fe.) & 0.14/0.14 & 0.14/0.13 & 0.14/0.13 & 0.17/0.15 & 0.15/0.13 & 0.15/0.14 \\
convnext(base-1k-224) & 0.14/0.14 & 0.25/0.25 & 0.20/0.20 & 0.13/0.14 & 0.15/0.17 & 0.19/0.18 \\
resnet50(cl.) & 0.15/0.14 & 0.14/0.15 & 0.16/0.14 & 0.18/0.17 & 0.17/0.16 & 0.16/0.14 \\
resnet50(fe.) & 0.13/0.14 & 0.13/0.15 & 0.16/0.16 & 0.13/0.12 & 0.18/0.18 & 0.15/0.15 \\
efficientnet(b0-cl.) & 0.14/0.14 & 0.17/0.18 & 0.17/0.18 & 0.18/0.17 & 0.18/0.17 & 0.21/0.20 \\
efficientnet(b0-fe.) & 0.16/0.15 & 0.18/0.16 & 0.19/0.19 & 0.22/0.20 & 0.19/0.18 & 0.20/0.19 \\
efficientnet(b1-cl.) & 0.12/0.12 & 0.12/0.13 & 0.12/0.11 & 0.13/0.13 & 0.13/0.13 & 0.13/0.12 \\
efficientnet(b1-fe.) & 0.13/0.12 & 0.14/0.13 & 0.14/0.12 & 0.13/0.13 & 0.14/0.13 & 0.15/0.13 \\
efficientnet(b4-cl.) & 0.15/0.14 & 0.13/0.14 & 0.13/0.12 & 0.15/0.15 & 0.17/0.15 & 0.16/0.16 \\
efficientnet(b4-fe.) & 0.13/0.15 & 0.19/0.19 & 0.15/0.15 & 0.

In [7]:
arr = np.array([[1, 2, 3],
                [4, 5, 6],
                [7, 8, 9]])

row_indices = np.array([0, 2])
col_indices = np.array([1, 2])

arr[np.ix_(row_indices, col_indices)] += 10

print(arr)


[[ 1 12 13]
 [ 4  5  6]
 [ 7 18 19]]


In [9]:
a = np.array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
rows = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0])
cols = np.arange(11)

In [10]:
a[np.ix_(rows, cols)] += 1

In [11]:
np.meshgrid()

[]

# Generating the synthetic dataset

In [1]:
import numpy as np

In [54]:
border_0 = 0.7
border_1 = 0.3

In [66]:
n_exp = 10000
# the target values
y = np.zeros(15)
y[9:] = 1

accs_1 = []
accs_2 = []

for i in range(n_exp):
  class_0 = np.random.uniform(0, border_0, size=(9, 3))
  class_1 = np.random.uniform(border_1, 1, size=(6, 3))
  X = np.concatenate([class_0, class_1], axis=0)
  X[:, 1] = np.concatenate([
    np.random.uniform(border_1, 1, size=(9, )), 
    np.random.uniform(0, border_0, size=(6, ))])
  
  # first case
  y_hat = np.sum(X, axis=1)
  # making the first 9 as the first class objects after sorting
  # other will be as the second class
  y_pred = np.ones(15)
  y_pred[np.argsort(y_hat)[:9]] = 0

  # accuracy
  acc = (y_pred == y).mean()
  accs_1.append(acc)

  # second case
  X_ = X.copy()
  X_[:, 1] *= -1
  y_hat = np.sum(X_, axis=1)
  # making the first 9 as the first class objects after sorting
  # other will be as the second class
  y_pred = np.ones(15)
  y_pred[np.argsort(y_hat)[:9]] = 0

  # accuracy
  acc = (y_pred == y).mean()
  accs_2.append(acc)

print(f"1st case: {np.mean(accs_1):.2f}, {np.std(accs_1):.2f}")
print(f"2nd case: {np.mean(accs_2):.2f}, {np.std(accs_2):.2f}")

1st case: 0.67, 0.12
2nd case: 0.91, 0.09


## Converting to latex

In [65]:
# this is one of the case since the process is randomness
for i in range(X.shape[1]):
  print(' & '.join([f'{val:.2f}' for val in X[:, i]]))

0.11 & 0.59 & 0.37 & 0.61 & 0.30 & 0.04 & 0.64 & 0.46 & 0.69 & 0.76 & 0.55 & 0.92 & 0.83 & 0.73 & 0.64
0.85 & 0.49 & 0.68 & 0.39 & 0.58 & 0.69 & 0.98 & 0.73 & 0.61 & 0.64 & 0.65 & 0.65 & 0.70 & 0.34 & 0.07
0.05 & 0.15 & 0.19 & 0.05 & 0.07 & 0.69 & 0.46 & 0.56 & 0.40 & 0.63 & 0.46 & 0.75 & 0.52 & 0.60 & 0.60
